# Augmentação das imagens

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image 
import pydicom
import os
import glob
import pickle
import pandas as pd
import csv
from pathlib import Path
from IPython.display import HTML
from os import listdir, mkdir
import imageio
from IPython import display
from sklearn.metrics import roc_auc_score, roc_curve, auc
import scipy

import tensorflow as tf
from keras.models import load_model
from keras import layers 
from keras.layers import Concatenate,Input, Dense, Activation, ZeroPadding2D, \
BatchNormalization, Flatten, Conv2D 
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, \
GlobalAveragePooling2D 
from keras.models import Model, Sequential 
from keras.preprocessing import image 
from keras.utils import layer_utils 
from keras.utils.data_utils import get_file 
from keras.applications.imagenet_utils import preprocess_input 
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [1]:
!pip install pydicom

You should consider upgrading via the 'c:\users\alex_\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
def augmentation(img):
   
   data_augmentation = tf.keras.Sequential([
     layers.experimental.preprocessing.RandomRotation(0.015)]) 
    
   imgA = np.zeros((9,500,500,3)) 
   mean = 0
   var = .005
   sigma = var ** 0.5
   gaussian = np.random.normal(mean, sigma, (500, 500,3)) 
   image = tf.expand_dims(img, 0)
   imgA[0,:,:,:] = tf.image.flip_left_right(img) 
   imgA[1,:,:,:] = augmented_image = data_augmentation(image)
   imgA[2,:,:,:] = augmented_image = data_augmentation(image)
   img2          = img + gaussian
   imgA[3,:,:,:] = img2
   image2 = tf.expand_dims(img2, 0)
   imgA[4,:,:,:] = tf.image.flip_left_right(img2) 
   imgA[5,:,:,:] = augmented_image = data_augmentation(image2)
   imgA[6,:,:,:] = augmented_image = data_augmentation(image2)
   imgA[7,:,:,:] = img - gaussian
   return imgA

In [3]:
# Abertura do CSV
dados = pd.read_csv('D:/ClassCSV/img_number.csv')
dados.head(15)

,sickness,csv,csv_entries,img_number
0,Aortic enlargement,aortic,7162,3067
1,Atelectasis,atelec,279,186
2,Calcification,calcif,960,452
3,Cardiomegaly,cardio,5427,2300
4,Consolidation,consol,556,353
5,ILD,ild,1000,386
6,Infiltration,infil,1247,613
7,Lung Opacity,lung,2483,1322
8,Nodule/Mass,nodu,2580,826
9,Other lesion,other,2203,1134


In [4]:
df_train  = pd.read_csv('D:/ClassCSV/pneumo.csv')
df_train.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,6e4391555899c8474c4d32f42b2ba21b,Pneumothorax,12,R10,1789.0,533.0,2189.0,948.0
1,e3c5ff1eb0d55039f3bbc3c4875543c3,Pneumothorax,12,R10,260.0,1240.0,1058.0,1622.0
2,54e6184c63c75a9695d7effc17969ad0,Pneumothorax,12,R10,244.0,521.0,896.0,1392.0
3,71511f823c4664475131c1519d9bd0a8,Pneumothorax,12,R9,1601.0,521.0,2537.0,2734.0
4,f9a0f4bd4bb3553af6e939362bbba5d6,Pneumothorax,12,R10,397.0,372.0,944.0,746.0


In [5]:
df_train.shape[0]

96

In [12]:
for i in range(0, df_train.shape[0]):
    os.link('D:/train/' + df_train['image_id'][i] + '.dicom',
            'D:/temporario/' + df_train['image_id'][i] + '.dicom')

In [6]:
os.chdir('D:/temporario/')
files=glob.glob("*.dicom")

#### Comandos de leitura e augmentação

In [7]:
mean = 0
var = .0125
sigma = var ** 0.5
gaussian = np.random.normal(mean, sigma, (500, 500,3)) 

In [8]:
Size   = df_train.shape[0]
sx, sy = 500,500
Xt     = np.zeros((sy,sx))
for j in range(0,Size):
    path = 'D:/temporario/' + files[j]
    dataset = pydicom.dcmread(path).pixel_array

    # normalize data
    Sig = np.max(dataset) - np.min(dataset)
    Med = np.average(dataset)
    img = (dataset-Med)/Sig
    
    # resize images
    xscale = sx/img.shape[0]
    yscale = sy/img.shape[1]
    img  = scipy.ndimage.interpolation.zoom(img, [xscale, yscale])

    # create 3 channels
    imgF = np.zeros((sy,sx,3))
    imgF[:,:,0] = img
    imgF[:,:,1] = img
    imgF[:,:,2] = img
    imgF = tf.image.flip_left_right(imgF)
    
    Aug = augmentation(imgF)
    
    # save data
    for i in range(4):
        path = 'D:/pickle1/' + files[j][:32] + '_' +str(i) + '.pckl'
        f = open(path, 'wb')
        pickle.dump(Aug[i,:,:,0], f)
        f.close()

c:\users\felipe\appdata\local\programs\python\python38\lib\site-packages\pydicom\pixel_data_handlers\pillow_handler.py:176: UserWarning: The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(
c:\users\felipe\appdata\local\programs\python\python38\lib\site-packages\pydicom\pixel_data_handlers\pillow_handler.py:176: UserWarning: The (0028,0101) 'Bits Stored' value (14-bit) doesn't match the JPEG 2000 data (16-bit). It's recommended that you change the 'Bits Stored' value
  warnings.warn(


In [9]:
os.chdir('D:/pickle1/')
files=glob.glob("*.pckl")

In [13]:
for j in range(0, 4*Size):
    path = 'D:/pickle1/' + files[j]
    f = open(path, 'rb')
    Y = pickle.load(f)
    
    # create 3 channels
    Y = np.zeros((sy,sx,3))
    Y[:,:,0] = img
    Y[:,:,1] = img
    Y[:,:,2] = img
    Y = tf.image.flip_left_right(imgF)
    
    Aug = augmentation(Y)
    
    # save data
    for i in range(8):
        path = 'D:/pickleteste/' + 'AUG_O_' + str(i) + '_' + files[j][:34] + '.pckl'
        f = open(path, 'wb')
        pickle.dump(Aug[i,:,:,0], f)
        f.close()

#### Remover arquivos para pasta temporária

In [16]:
for i in range(0, df_train.shape[0]):
    os.remove('D:/temporario/' + df_train['image_id'][i] + '.dicom')

In [ ]:
targets = pd.read_csv('D:/ClassCSV/train-multiple-targets.csv')
targets.head()

In [ ]:
train = pd.read_csv('D:/ClassCSV/train.csv')
train.head()

In [39]:
# Encontrar as informações referentes a uma determinada imagem
id_imagem = '47ed17dcb2cbeec15182ed335a8b5a9e'
targets.loc[(targets["image_id"] == id_imagem)]

,Unnamed: 0,image_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
7,7,47ed17dcb2cbeec15182ed335a8b5a9e,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0


In [38]:
train.loc[(train["image_id"] == id_imagem)]

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
7,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0
1229,47ed17dcb2cbeec15182ed335a8b5a9e,Other lesion,9,R9,557.0,2352.0,675.0,2484.0
2771,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R8,544.0,2345.0,674.0,2485.0
20419,47ed17dcb2cbeec15182ed335a8b5a9e,Other lesion,9,R9,1715.0,906.0,2054.0,1387.0
48000,47ed17dcb2cbeec15182ed335a8b5a9e,Other lesion,9,R10,1723.0,983.0,2048.0,1398.0
51769,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R10,562.0,2346.0,663.0,2460.0
57406,47ed17dcb2cbeec15182ed335a8b5a9e,Lung Opacity,7,R8,327.0,937.0,1231.0,2739.0
64879,47ed17dcb2cbeec15182ed335a8b5a9e,Lung Opacity,7,R10,254.0,812.0,1193.0,2978.0
